sample.csv 에는 일단 300개 데이터셋만 저장

모든 함수는 악성 가능성이 있으면 1, 아니면 0
ex) // 가 중간에 포함되면 1, 아니면 0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os

data = pd.read_csv('/content/drive/MyDrive/Epro/url_dataset.csv')

data.head()

,id,url
0,1,google.com
1,2,youtube.com
2,3,facebook.com
3,4,baidu.com
4,5,wikipedia.org


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   phish_id           299 non-null    int64 
 1   url                299 non-null    object
 2   phish_detail_url   299 non-null    object
 3   submission_time    299 non-null    object
 4   verified           299 non-null    object
 5   verification_time  299 non-null    object
 6   online             299 non-null    object
 7   target             299 non-null    object
dtypes: int64(1), object(7)
memory usage: 18.8+ KB


## 전처리

In [3]:
#url 길이

def length(url):
  return len(url)

In [4]:
#url 단축 여부
import requests

def url_shorten(url):
    req = 'https://unshorten.me/json/'+url
    response = requests.request("GET", req).json()
    if (response['success'] and response['resolved_url'] != url):    # 단축 url인 경우 -> 피싱
        return 1
    else:                                                            # 정상
        return 0

In [5]:
#@여부

def at_sign(url):
  if '@' in url:
    return 1
  else:
    return 0

In [6]:
# 중간에 // 여부, redirection 확인

def double_slash(url):
  if '//' in url[7:]:
    return 1
  else:
    return 0

In [7]:
#점 개수

def dot(url):
  return url.count('.')

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=is_king&logNo=221520718981

In [8]:
#IP주소
from urllib.parse import urlparse

def IP_add(url):
  parse = urlparse(url)
  domain = parse.netloc.split('.')
  try:
    int(domain[-1],0)
    return 1
  except ValueError:
    return 0

In [9]:
# - 있는지

def dash(url):
  if '-' in url:
    return 1
  else:
    return 0

In [10]:
#http 인지 https 인지 확인

def http(url):
  if url.startswith('http://'):
    return 1
  else:
    return 0

In [11]:
file = pd.read_csv('/content/drive/MyDrive/Epro/topsites.csv')

file.head()

,topsites
0,google.com
1,youtube.com
2,facebook.com
3,baidu.com
4,wikipedia.org


In [12]:
list = []
for index in file['topsites']:
  list.append(index.split('.')[0].lower())

print(list)

['google', 'youtube', 'facebook', 'baidu', 'wikipedia', 'yahoo', 'google', 'reddit', 'qq', 'amazon', 'taobao', 'twitter', 'tmall', 'google', 'live', 'vk', 'instagram', 'sohu', 'sina', 'jd', 'weibo', '360', 'google', 'google', 'linkedin', 'google', 'list', 'google', 'google', 'yandex', 'netflix', 'google', 'yahoo', 'google', 't', 'ebay', 'google', 'pornhub', 'imgur', 'bing', 'msn', 'onclkds', 'twitch', 'google', 'google', 'tumblr', 'gmw', 'alipay', 'livejasmin', 'xvideos', 'mail', 'microsoft', 'ok', 'aliexpress', 'wordpress', 'hao123', 'stackoverflow', 'imdb', 'github', 'amazon', 'blogspot', 'pinterest', 'csdn', 'wikia', 'apple', 'google', 'popads', 'youth', 'office', 'bongacams', 'paypal', 'microsoftonline', 'google', 'google', 'whatsapp', 'google', 'xhamster', 'detail', 'diply', 'google', 'adobe', 'coccoc', 'craigslist', 'nicovideo', 'txxx', 'dropbox', 'amazon', 'google', 'amazon', 'googleusercontent', 'google', 'booking', 'thepiratebay', 'google', 'porn555', 'kbstar', 'shinhan', 'woo

In [ ]:
#이미 사용되고있는 사이트명 / 브랜드명을 사용하고 있는지 -> alexa 등에서 저장하기

def brands_without_similarity(url):
  domain = urlparse(url).netloc.lower()
  for names in list:
    if names in domain:
      return 1
    else:
      return 0

In [48]:
#브랜드명 유사도 비교해서 검색

def ngram(s, num):  #num : 몇글자씩 끊을 건지
    res=[]
    slen=len(s)-num+1 # slen : 끊었을 때 나오는 개수
    for i in range (slen):
        ss=s[i:i+num] #num만큼 s문자열에서 단어 자르기
        res.append(ss) #자른 단어는 res배열에 저장
    return res

def diff_ngram(sa,sb,num):
    a=ngram(sa,num)	#a문자열을 num단어씩 자른 배열
    b=ngram(sb,num) #b문자열을 num단어씩 자른 배열
    r=[]
    cnt=0
    for i in a:
        for j in b:
            if i==j:    #a에서 자른 단어가 b에도 있다면
                cnt+=1   #cnt++
                r.append(i) #중복되는 단어i를 r배열에 추가한다.
    return cnt/len(b) #cnt/len=(중복되는 횟수/b의 길이), r=중복되는 단어

def brands(url):
  domain = urlparse(url).netloc.lower()
  for names in list:
    return diff_ngram(domain, names, 2)

In [45]:
# example = 'https://azamzon.zapto.org/'

# res = diff_ngram(example, 'amazon',2)

# print(res)

(0.8, ['am', 'ma', 'az', 'zo', 'on'], ['az', 'am', 'zo', 'on'])


In [50]:
res = []
index = 0

for url in data['url']:
  res.append([])
  res[index].append(length(url))
  res[index].append(url_shorten(url))
  res[index].append(at_sign(url))
  res[index].append(double_slash(url))
  res[index].append(dot(url))
  res[index].append(IP_add(url))
  res[index].append(dash(url))
  res[index].append(http(url))
  res[index].append(brands(url))

  index+=1

In [51]:
for i in range(10):
  print(res[i])

[10, 0, 0, 1, 0, 0, 0, 0.0]
[11, 0, 0, 1, 0, 0, 0, 0.0]
[12, 0, 0, 1, 0, 0, 0, 0.0]
[9, 0, 0, 1, 0, 0, 0, 0.0]
[13, 0, 0, 1, 0, 0, 0, 0.0]
[9, 0, 0, 1, 0, 0, 0, 0.0]
[12, 0, 0, 2, 0, 0, 0, 0.0]
[10, 0, 0, 1, 0, 0, 0, 0.0]
[6, 0, 0, 1, 0, 0, 0, 0.0]
[10, 0, 0, 1, 0, 0, 0, 0.0]


dataframe

In [52]:
columns = ['length', 'url_shorten', 'at_sign', 'redirection', 'dot', 'IP', 'dash', 'http', 'brands']
final_res = pd.DataFrame(res, columns= columns)

In [53]:
final_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14038 entries, 0 to 14037
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   length       14038 non-null  int64  
 1   at_sign      14038 non-null  int64  
 2   redirection  14038 non-null  int64  
 3   dot          14038 non-null  int64  
 4   IP           14038 non-null  int64  
 5   dash         14038 non-null  int64  
 6   http         14038 non-null  int64  
 7   brands       14038 non-null  float64
dtypes: float64(1), int64(7)
memory usage: 877.5 KB


In [54]:
final_res.head()

,length,at_sign,redirection,dot,IP,dash,http,brands
0,10,0,0,1,0,0,0,0.0
1,11,0,0,1,0,0,0,0.0
2,12,0,0,1,0,0,0,0.0
3,9,0,0,1,0,0,0,0.0
4,13,0,0,1,0,0,0,0.0


In [55]:
final_res.describe()

,length,at_sign,redirection,dot,IP,dash,http,brands
count,14038.000000,14038.000000,14038.000000,14038.000000,14038.000000,14038.00000,14038.000000,14038.000000
mean,37.665978,0.016598,0.004132,1.792349,0.007622,0.23586,0.215415,0.033010
std,55.547910,0.127763,0.064147,1.870337,0.086975,0.42455,0.411125,0.140086
min,4.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
25%,12.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
50%,20.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
75%,45.000000,0.000000,0.000000,2.000000,0.000000,0.00000,0.000000,0.000000
max,2631.000000,1.000000,1.000000,148.000000,1.000000,1.00000,1.000000,1.200000


In [56]:
final_res.to_csv('/content/drive/MyDrive/Epro/final_preprocessed_data.csv')